<a href="https://colab.research.google.com/github/woo2201/ColabDrive/blob/reservior/JobMarket_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
cd /content/gdrive/MyDrive/dataset/JobMarket/

/content/gdrive/MyDrive/dataset/JobMarket


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import lxml
import re
import html
import urllib

In [4]:
pip install konlpy

In [5]:
from konlpy.tag import Okt
okt = Okt()

링크 : http://comp.fnguide.com/SVO2/ASP/SVD_UJRank.asp

해당 링크에는 500대 한국 기업의 전반적인 재무재표성 데이터가 있습니다.
상위 500개 기업을 나열해서, 매출액 및 자산규모/매출액과 채용 규모 및 만족도에 어떠한 관계가 있는지 살펴보려 합니다.

1. 우리의 고용자가 될 수 있는 500대 기업의 정보에 대해 긁어옵니다.
물론 더욱 다양한 기업들이 있습니다만, 많은 구직자들이 

In [8]:
companies = pd.read_csv('companyInfo.csv')

In [11]:
companies.loc[2, '종목명'] = '한국전력공사'

In [15]:
class infoParser:
    def __init__(self, company):
        self.company = company
        try:
            self.html = urlopen("https://www.teamblind.com/kr/company/" + urllib.parse.quote(str(self.company)))  
            self.bsObject = BeautifulSoup(self.html, "html.parser", from_encoding='utf-8')
        except urllib.error.HTTPError as err:
            pass     

    def description(self):
        try:
            val = self.bsObject.find('div', {"class":"desc all"}).text
        except AttributeError as e:
            return None
        return val

    def overallRating(self):
        try:
            val = self.bsObject.find('span', {"class" : "star"}).text
        except AttributeError as e:
            return None
        return val

    def individualStar(self):
        try:
            val = [float(i.text) for i in self.bsObject.find_all('strong',{'class' : "star"})]
        except AttributeError as e:
            return None
        return val
    
    def reviewCount(self):
        try:
            val = self.bsObject.find('em',{'class' : "count"}).text
        except AttributeError as e:
            return None
        return val

    def industryCategory(self):
        try:
            val = self.bsObject.select_one('#wrap > section > div > div > div.cpctw.cpovw > div > div > div > section.overview > ul > li:nth-of-type(2)').text
        except AttributeError as e:
            return None
        return val 


In [16]:
df = pd.DataFrame({'company' : [],
                   'description': [],
                   'overallRating': [],
                   'individualStar' : [],
                   'reviewCount': [],
                   'industryCat': []})  # starName = [str(i.text) for i in bsObject.find_all('span', {'class' : 'desc'})][0:5]; starName

for cpny in companies['종목명']:
    test = infoParser(company = cpny)
    adix = {'company' : test.company,
            'description': test.description(),
            'overallRating': test.overallRating(),
            'individualStar' : test.individualStar(),
            'reviewCount': test.reviewCount(),
            'industryCat': test.industryCategory()}
    
    df = df.append(adix, ignore_index = True)


In [17]:
df.sample(5)

,company,description,overallRating,individualStar,reviewCount,industryCat
205,피아이첨단소재,"피아이첨단소재는 PI Film 전문기업입니다. 디스플레이, 반도체, 전기자동차 등에...",Rating Score3.5,"[3.5, 3.4, 4.3, 2.8, 2.5]",49개 리뷰,업계\n 기타 제조/수리업\n
479,두산중공업,"두산중공업은 두산그룹 계열사로 기관, 터빈, 선박용엔진 등의 생산 및 공급을 주요 ...",Rating Score2.9,"[2.8, 4.2, 2.5, 3.1, 2.0]",454개 리뷰,업계\n 건설/부동산업\n
26,미래에셋대우,"미래에셋대우는 글로벌 비즈니스 투자 플랫폼 구축, 금융 관련 종합 솔루션 제시, 다...",Rating Score3.4,"[3.4, 3.4, 3.2, 2.7, 2.5]",279개 리뷰,업계\n 금융업\n
433,에스엠,접기,Rating Score0.0,[],None,업계\n 기타 제조/수리업\n
294,네패스,"네패스는 봉사하는 생활, 도전하는 자세, 감사하는 마음이라는 경영이념 아래 미래 사...",Rating Score2.9,"[2.9, 3.0, 2.2, 2.9, 2.3]",261개 리뷰,업계\n 반도체/전자기기 제조업 \n


In [18]:
df = df[~df.description.isnull()]

In [19]:
df['overallRating'] = df.overallRating.apply(lambda x : re.sub('[a-zA-z]','',x))

In [20]:
df['careerImprovement'] = df.individualStar.apply(lambda x: x[0] if x else '')
df['workLifeBalance'] = df.individualStar.apply(lambda x: x[1] if x else '')
df['salaryWelfare'] = df.individualStar.apply(lambda x: x[2] if x else '')
df['workplaceCulture'] = df.individualStar.apply(lambda x: x[3] if x else '')
df['boardMembers'] = df.individualStar.apply(lambda x: x[4] if x else '')

,company,description,overallRating,individualStar,reviewCount,industryCat,careerImprovement,workLifeBalance,salaryWelfare,workplaceCulture,boardMembers
0,삼성전자,"삼성전자는 삼성그룹 계열사로 휴대폰, 컴퓨터, 네트워크시스템, 반도체부품, 디스플레...",3.5,"[3.2, 3.1, 3.7, 3.2, 2.6]",7097,업계\n 반도체/전자기기 제조업 \n,3.2,3.1,3.7,3.2,2.6
1,SK하이닉스,"SK하이닉스는 반도체 전문기업으로 반도체, 컴퓨터, 통신기기 제조 등의 사업을 영위...",2.3,"[2.2, 2.3, 2.5, 2.2, 1.8]",3277,업계\n 반도체/전자기기 제조업 \n,2.2,2.3,2.5,2.2,1.8
2,한국전력공사,한국전력공사는 국민경제 발전 기여를 목적으로 설립된 법인으로 시장형 공기업에 해당합...,4.0,"[3.4, 4.2, 3.9, 3.1, 3.0]",1668,업계\n 국가/공공기관\n,3.4,4.2,3.9,3.1,3
3,SK,"SK는 에너지, 화학, 정보통신, 반도체, 마케팅, 바이오, 물류 부문에서 사업을 ...",3.5,"[3.1, 4.6, 3.2, 3.7, 2.8]",14,업계\n 금융업\n,3.1,4.6,3.2,3.7,2.8
4,SK이노베이션,"SK이노베이션은 에너지, 화학 분야를 선도하는 기업으로 석유개발, 정유, 화학, 윤...",3.9,"[3.6, 3.9, 3.8, 3.9, 3.3]",729,업계\n 석유정제품 제조업\n,3.6,3.9,3.8,3.9,3.3
...,...,...,...,...,...,...,...,...,...,...,...
475,LG디스플레이,"LG디스플레이는 OLED, Curved OLED 등의 제조 기술력을 보유하고 있는 ...",2.9,"[2.8, 2.9, 2.8, 2.7, 2.3]",1659,업계\n 반도체/전자기기 제조업 \n,2.8,2.9,2.8,2.7,2.3
476,S-Oil,S-OIL은 수익성 있는 지속성장을 미션으로 하는 정유/석유화학 업계의 대표회사입니...,3.4,"[2.7, 3.7, 3.8, 2.7, 2.1]",212,업계\n 석유정제품 제조업\n,2.7,3.7,3.8,2.7,2.1
477,롯데쇼핑,"롯데쇼핑은 롯데그룹의 계열사로 백화점, 할인점, 전자제품전문점, 슈퍼, TV홈쇼핑 ...",2.6,"[2.4, 2.9, 2.4, 2.2, 1.8]",807,업계\n 종합 소매업\n,2.4,2.9,2.4,2.2,1.8
478,삼성중공업,삼성중공업은 강선 건조업을 주 업으로 하는 삼성그룹의 계열사입니다. 다양한 종류의 ...,2.1,"[2.2, 3.3, 1.8, 2.1, 1.4]",746,업계\n 조선업\n,2.2,3.3,1.8,2.1,1.4


In [23]:
df['reviewCount'] = df['reviewCount'].apply(lambda x: re.sub('[a-zA-z가-힣]','',x) if x else '')
df['reviewCount'] = df['reviewCount'].apply(lambda x: x.replace(',','')).apply(lambda x: int(x) if x else 0)

TypeError: ignored

In [ ]:
df['repNouns'] = df.description.apply(lambda x: okt.nouns(x))

In [29]:
df['industry'] = df.industryCat.apply(lambda x: x.replace('업계','').replace('\n', ''))

In [31]:
df = df.drop(['description','individualStar','industryCat'], axis = 1)

In [35]:
secondary = df[['company','overallRating','repNouns']].explode('repNouns')

In [36]:
df.to_csv('/content/gdrive/MyDrive/dataset/JobMarket/completeList.csv')

In [37]:
secondary.to_csv('/content/gdrive/MyDrive/dataset/JobMarket/completeList_second.csv')

In [108]:
#chk.bsObject.find_all(attrs = {'class' : 'sctit'})
# [child for child in chk.bsObject.ul.children]
# tip : error 해결. https://mjdeeplearning.tistory.com/46

[<h1 class="sctit">삼성전자 회사소개</h1>,
 <h2 class="sctit">삼성전자 리뷰</h2>,
 <h2 class="sctit">삼성전자 게시글</h2>,
 <h2 class="sctit">삼성전자 뉴스</h2>,
 <h2 class="sctit">삼성전자 연봉 정보</h2>,
 <h2 class="sctit">삼성전자 채용 공고</h2>]